In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ordpy as ordpy
%matplotlib inline
import os
import numpy as np
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.cm as cm
from cycler import cycler

In [27]:
df = pd.read_csv('../raw_data/outbreaks_disease_location.csv')

In [3]:
len(df['disease'].unique())

13

In [4]:
all_locs = df['location'].unique()
len([x for x in all_locs if len(x)!=2])

245

In [5]:
len(df[['disease','location','event']].drop_duplicates())

587

### Entropy Calculation

In [7]:
df_data = df[[x for x in df.columns if x.isdigit()]].T
df_norm = df_data.divide(df_data.sum(),axis=1)

In [39]:
df_entropy = df_norm.fillna(0).apply(lambda x: (-x*np.log(x)/np.log(2))).sum()
# df_entropy.hist(bins=100)

/apps/software/standard/core/jupyterlab/4.4.6-py3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


cols=['unique_id','disease','location','event','start_date','end_date','duration','total']


In [18]:
df_entropy=df[cols].merge(pd.DataFrame(df_entropy).rename(columns={0:'entropy'}),right_index=True,left_index=True)

In [21]:
df_entropy.to_csv('output/entropy.csv',index=None)

### Permutation Entropy Calculation

In [28]:
df=df.set_index('unique_id')

In [23]:
df_data = df[[x for x in df.columns if x.isdigit()]]


In [24]:
for i in df_data.index:
    y=df_data.loc[i].dropna().values
    pe=ordpy.permutation_entropy(y, dx=3, taux=1, normalized=True)
    df_data.loc[i,'PE']=pe

/tmp/ipykernel_373901/907780680.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.loc[i,'PE']=pe


In [36]:
df_pe=df.reset_index()[cols].merge(df_data['PE'].reset_index())

In [38]:
df_pe.to_csv('output/permutation_entropy.csv',index=None)

### Kurtosis and Skewness

In [42]:
import numpy as np

def pdf_moments(x, p):
    """
    Moments of a discrete PDF.

    Parameters
    ----------
    x : 1D array
        Support (time index, epiweek, etc.)
    p : 1D array
        Nonnegative weights (will be normalized to sum to 1)

    Returns
    -------
    dict with mean, median, variance, skewness, excess_kurtosis
    """
    x = np.asarray(x, dtype=float)
    p = np.asarray(p, dtype=float)

    # Defensive normalization
    p = p / p.sum()

    # ---- Mean ----
    mu = np.sum(x * p)

    # ---- Median (weighted median) ----
    order = np.argsort(x)
    x_sorted = x[order]
    p_sorted = p[order]
    cdf = np.cumsum(p_sorted)

    median = x_sorted[np.searchsorted(cdf, 0.5)]

    # ---- Variance and higher moments ----
    var = np.sum((x - mu)**2 * p)
    sigma = np.sqrt(var)

    if sigma < 1e-12:
        skew = np.nan
        kurtosis_excess = np.nan
    else:
        skew = np.sum((x - mu)**3 * p) / sigma**3
        kurtosis_excess = np.sum((x - mu)**4 * p) / sigma**4 - 3

    return {
        "mean": [mu],
        "median": [median],
        "variance": [var],
        "skewness": [skew],
        "excess_kurtosis": [kurtosis_excess]
    }


In [49]:
df_data = df[[x for x in df.columns if x.isdigit()]]

df_norm=df_data.div(df_data.sum(axis=1), axis=0)


In [50]:
all_stats_df=pd.DataFrame()
for i in df_norm.index:
    probs=df_norm.loc[i].dropna()
    supp=[float(x) for x in probs.index]
    stat=pdf_moments(supp,probs.values)
    stat['unique_id']=i
    statdf=pd.DataFrame(stat)
    all_stats_df=pd.concat([all_stats_df,statdf])

In [60]:
df_shape=df.reset_index()[cols].merge(all_stats_df)

In [58]:
df_shape

,unique_id,disease,location,event,start_date,end_date,duration,total
0,1,RSV,ND,HOSPITALIZATIONS,2024-06-22 00:00:00,2025-05-31 00:00:00,41,459.000000
1,2,ILI,NEBRASKA,PERCENT_UNWEIGHTED,2010-10-16 00:00:00,2011-08-06 00:00:00,34,90.239022
2,3,ILI,NEBRASKA,PERCENT_UNWEIGHTED,2012-06-16 00:00:00,2012-12-08 00:00:00,17,54.807999
3,4,ILI,NEBRASKA,PERCENT_UNWEIGHTED,2012-10-13 00:00:00,2013-08-03 00:00:00,34,153.295142
4,5,ILI,NEBRASKA,PERCENT_UNWEIGHTED,2013-06-08 00:00:00,2013-12-21 00:00:00,20,42.587176
...,...,...,...,...,...,...,...,...
10794,10795,ILI,MINNESOTA,PERCENT_UNWEIGHTED,2020-12-12 00:00:00,2021-08-07 00:00:00,26,12.590682
10795,10796,ILI,MINNESOTA,PERCENT_UNWEIGHTED,2021-06-12 00:00:00,2022-04-23 00:00:00,37,95.250752
10796,10797,ILI,MINNESOTA,PERCENT_UNWEIGHTED,2022-02-26 00:00:00,2022-08-20 00:00:00,17,26.389983
10797,10798,ILI,MINNESOTA,PERCENT_UNWEIGHTED,2023-07-22 00:00:00,2024-08-24 00:00:00,49,63.940555
